## Importing Requrired Packages

In [1]:
import pandas as pd
import numpy as np
import tweepy
from tweepy import OAuthHandler
from textblob import TextBlob
import datetime
from dateutil.parser import parse
from collections import Counter
import pickle
import re

## TweePy Keys and Tokens

In [ ]:
consumer_key =  'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'
consumer_secret = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'
access_token = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'
access_token_secret = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'

 ## Create OAuthHandler object

In [ ]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

## Create tweepy API object to fetch tweets 

In [ ]:
api = tweepy.API(auth)

# PreProcessing 

In [ ]:
def preprocess_tweet(tweet):
    #Preprocess the text in a single tweet
    #arguments: tweet = a single tweet in form of string 
    #handling emojis 
    tweet = emoji.demojize(tweet)
    tweet = tweet.replace(":"," ")
    #convert the tweet to lower case
    tweet = tweet.lower()
    #convert all urls to sting "URL"
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL',tweet)
    #convert all @username to "AT_USER"
    tweet = re.sub('@[^\s]+','AT_USER', tweet)
    #correct all multiple white spaces to a single white space
    tweet = re.sub('[\s]+', ' ', tweet)
    #convert "#topic" to just "topic"
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
    return tweet

# Sentiment Analysis Using TextBlob

In [ ]:
def get_tweet_sentiment(tweet): 
    ''' tweet
    Utility function to classify sentiment of passed tweet 
    using textblob's sentiment method 
    '''
    # create TextBlob object of passed tweet text 
    analysis = TextBlob(preprocess_tweet(tweet)) 
    # set sentiment 
    if analysis.sentiment.polarity > 0: 
        return 'Positive'
    elif analysis.sentiment.polarity == 0: 
        return 'Nuetral'
    else: 
        return 'Negative'

# Fetching Tweets

In [ ]:
dates = pd.date_range(start='2/25/2020',end='3/3/2020',freq='D')

 ## Fetching Tweets for Particular Dates

In [ ]:
All_tweets = []
for date in dates:
    fetched_tweets = api.search(q = Search, count = 200 ,until = date.date()) 
    All_tweets.append(fetched_tweets)

In [ ]:
new_sent = {}
sentiment = {}

for fetched_tweets in All_tweets:
    for date in dates:
        date_sentiment = []
        for tweet in fetched_tweets: 
             # empty dictionary to store required params of a tweet 
             #date_sentiment = [] 
            #print(tweet.created_at)
             ## Processing
            if tweet.created_at.strftime('%Y-%m-%d') == str(date.date()):
                date_sentiment.append(get_tweet_sentiment(tweet.text)) 
                new_sent[date.date().strftime('%Y-%m-%d')] = Counter(date_sentiment).most_common(1)
                #print(date,get_tweet_sentiment(tweet.text))

In [ ]:
Tweet_sentiment = pd.DataFrame(new_sent).transpose()

In [ ]:
Tweet_sentiment